© 2024 Luca Kunz. Commercial use is subject to the terms of the source repository's license. All other commercial rights are reserved.

# Produce a statistics dataframe about (hyperbolic) drifter-TRAP pairs

We first load the PROXIMITY, the PTRT and the TRAPS GPGP pkl files. Then we group all drifter rows by their pair ID in order to obtain a dataframe with one drifter-TRAP pair per row. Each pair will be assigned a range of old and new attributes like e.g. the lifetime of the associated TRAP, the TRAP age at first encounter, an indicator if measurements of the vorticity curve are available for all pair instances as well as arrays of the core attractions and quadrupole orders a pair involves. With this new dataframe we can analyse the characteristics of drifter-TRAP pairs from many perspectives and produce statistics. Since, among other things, we are looking for the conditions that cause hyperbolic drifter motion, we call this final dataframe **hyperbolic pair statistics (HPS)**.

In [1]:
import os
import sys
import numpy as np
from scipy.interpolate import interp1d
from scipy.stats import pearsonr
import pandas as pd
import time
import datetime
import pickle

from IPython.display import display, Audio
# import jupyter notebook files like regular python modules
import import_ipynb
from aa_define_classes import TRAPSdata

importing Jupyter notebook from aa_define_classes.ipynb


In [2]:
# measure the computation time for the entire script
start_script_timer = time.perf_counter()

# Preliminary

## Pickle object saving function

In [3]:
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

## Files and paths

In [4]:
# if script is running in jupyter lab
if sys.argv[0].endswith("ipykernel_launcher.py"):
    # set the velocity product
    vel_product_ID = 1
    epsilon_ID = 1
    rho_ID = 1
    drogue_ID = 1
    notebook_run = True
#    save_fig = True
    save_fig = False

    
# if script is running as python script
else:
    # read in product from bash
    vel_product_ID = int(sys.argv[1])
    # read in epsilon from bash
    epsilon_ID = int(sys.argv[2])
    # read in rho from bash
    rho_ID = int(sys.argv[3])
    # read in drogue state from bash
    drogue_ID = int(sys.argv[4])
    notebook_run = False
    save_fig = True


vel_product_short = ['ENSRYS_24HM', 'MULTIOBS_24HI', 'MULTIOBS_24HM', 'SEALEVEL_24HI'][vel_product_ID]

vel_product_long = ['CMEMS GLOBAL_REANALYSIS_PHY_001_031 ENSEMBLE MEAN (1/4°, 24HM)', 
                    'CMEMS MULTIOBS_GLO_PHY_REP_015_004 (1/4°, 24HI)', 
                    'CMEMS MULTIOBS_GLO_PHY_REP_015_004 (1/4°, 24HM)', 
                    'CMEMS SEALEVEL_GLO_PHY_L4_MY_008_047 (1/4°, 24HI)'][vel_product_ID]


years = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', 
         '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']

epsilon_value = [0.1, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0][epsilon_ID]
epsilon_string = ['_e010', '_e025', '_e050', '_e075', '_e100', '_e125', '_e150', '_e175', '_e200'][epsilon_ID]

rho_kmvalue = [50, 75, 100, 150, 200, 250, 300][rho_ID] # in kilometres
rho_kmstring = ['_r050', '_r075', '_r100', '_r150', '_r200', '_r250', '_r300'][rho_ID] # in kilometres

gdp_product_ID = 2 # set this apriori
gdp_product_short = ['GDP_1HI', 'GDP_6HI', 'GDP_24HI'][gdp_product_ID]

drogue_state = ['_ALL', '_DROGUED', '_UNDROGUED'][drogue_ID]

In [5]:
# define the path to the pkl files
pkl_PROXIMITY_importpath = 'export_pkl/' + vel_product_short + '/'
pkl_VIRCLES_importpath = 'export_pkl/' + vel_product_short + '/20XX/'
pkl_HPS_exportpath = 'export_pkl/' + vel_product_short + '/'

# Import PROXIMITY dataframe

In [6]:
# define the pkl files to load
pkl_PROXIMITY_importname = vel_product_short + epsilon_string + '_' + gdp_product_short
pkl_PROXIMITY_importname += drogue_state + '_TRAPS_DRIFTERS_PROXIMITY_0019' + rho_kmstring + '.pkl'

# load the pickle object
with open(pkl_PROXIMITY_importpath + pkl_PROXIMITY_importname, 'rb') as inp:
    # cdf for current DataFrame, this will be overwritten every loop to save memory
    pd_PROXIMITY_df = pickle.load(inp).pd_TRAPS_df.copy()
    
# reset the index after loading and cropping
pd_PROXIMITY_df.reset_index(drop=True, inplace=True)

In [7]:
# print check
pd_PROXIMITY_df

,drifter_ID,time,drifter_lat,drifter_lon,drifter_U,drifter_V,drogued,TRAP_ID,distance_to_core,core_lon,...,age,pair_ID,proximity_time,proximity_age,drifter_vector_lon,drifter_vector_lat,drifter_angle_DEG,drifter_vector_zonal,drifter_vector_meridional,drifter_angle_KM
0,9936,2000-08-26,22.555,-155.354,-0.23282,0.25778,True,200008250000 003,60.231203,-155.817593,...,2,200008260000 000000000009936 200008250000 003,48,24,0.463593,-0.333091,5.660152,47.516605,-37.013106,5.621412
1,9936,2000-08-27,22.751,-155.505,-0.14815,0.24477,True,200008250000 003,45.624820,-155.920540,...,3,200008260000 000000000009936 200008250000 003,48,48,0.415540,-0.147969,5.941097,42.559063,-16.442333,5.914509
2,9936,2000-08-28,22.937,-155.619,-0.12629,0.22759,True,200008280000 005,63.503220,-155.038057,...,1,200008280000 000000000009936 200008280000 005,24,24,-0.580943,-0.201953,3.476154,-59.405903,-22.440980,3.502778
3,9936,2000-08-29,23.084,-155.726,-0.11437,0.14587,True,200008160000 382,63.939969,-156.332764,...,14,200008290000 000000000009936 200008160000 382,144,24,0.606764,-0.140932,6.054965,61.992535,-15.660309,6.035746
4,9936,2000-08-30,23.217,-155.862,-0.18390,0.16488,True,200008160000 382,52.500023,-156.373184,...,15,200008290000 000000000009936 200008160000 382,144,48,0.511184,0.049388,0.096315,52.212402,5.487939,0.104723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57558,300234066815360,2019-12-14,23.648,-148.937,-0.08284,-0.21683,True,201912110000 022,39.160872,-148.552352,...,4,201912130000 300234066815360 201912110000 022,144,48,-0.384648,0.006660,3.124279,-39.153878,0.740097,3.122693
57559,300234066815360,2019-12-15,23.512,-148.918,-0.04516,-0.22107,True,201912110000 022,32.657723,-148.632829,...,5,201912130000 300234066815360 201912110000 022,144,72,-0.285171,-0.134407,3.582036,-29.042440,-14.935311,3.616582
57560,300234066815360,2019-12-16,23.355,-148.910,0.06173,-0.21849,True,201912110000 022,41.771054,-148.603006,...,6,201912130000 300234066815360 201912110000 022,144,96,-0.306994,-0.249044,3.823147,-31.288691,-27.673792,3.865759
57561,300234066815360,2019-12-17,23.188,-148.857,0.01730,-0.28545,True,201912110000 022,53.748656,-148.684404,...,7,201912130000 300234066815360 201912110000 022,144,120,-0.172596,-0.457034,4.351302,-17.599340,-50.785640,4.378798


# Import PTRT dataframe

In [8]:
# define the pkl files to load
pkl_PTRT_importname = vel_product_short + epsilon_string + '_' + gdp_product_short
pkl_PTRT_importname +=  drogue_state + '_TRAPS_DRIFTERS_PTRT_0019' + rho_kmstring + '.pkl'

# load the pickle object
with open(pkl_PROXIMITY_importpath + pkl_PTRT_importname, 'rb') as inp:
    # cdf for current DataFrame, this will be overwritten every loop to save memory
    pd_PTRT_df = pickle.load(inp).pd_TRAPS_df.copy()
        
# reset the index after loading
pd_PTRT_df.reset_index(drop=True, inplace=True)

In [9]:
# extract origin IDs from pair IDs
pd_PTRT_df.insert(1, 'origin_ID', [cpair_ID[-16:] for cpair_ID in pd_PTRT_df.pair_ID])

In [10]:
# print check
pd_PTRT_df

,pair_ID,origin_ID,counts,proximity_time
0,201310310000 000000000101916 201310270000 344,201310270000 344,46,1104
1,201905270000 300234066316870 201905150000 291,201905150000 291,45,1080
2,201507280000 000000000132718 201507280000 374,201507280000 374,44,1056
3,200010120000 000000009729746 200010120000 069,200010120000 069,41,984
4,201102160000 000000000093582 201101310000 284,201101310000 284,39,936
...,...,...,...,...
16804,200912120000 000000000079281 200912100000 524,200912100000 524,1,24
16805,200912130000 000000000079271 200911250000 277,200911250000 277,1,24
16806,200912130000 000000000079281 200911250000 268,200911250000 268,1,24
16807,200912170000 000000000079281 200912170000 581,200912170000 581,1,24


# Import TRAPS GPGP DataFrame

Load the yearly TRAPS GPGP dataframes and stack them together.  
Since we actually do this to get the attributes related to the vorticity circle, we flag all related variables `_VIRCLE` for clarity.

In [11]:
# the lists to store the subdataframes to feed into pd.concat() after the loop
pd_VIRCLES_sdfs = []

for year in years:
    
    start_timer = time.perf_counter()
    
    # define the pkl files to load
    pkl_VIRCLES_importname = vel_product_short + epsilon_string + '_TRAPS_GPGP_' + year + '.pkl'

    # load the pickle objects
    with open(pkl_VIRCLES_importpath + pkl_VIRCLES_importname, 'rb') as inp:
        
        # selecting attributes saves memory and speeds up the loading
        pd_VIRCLES_sdfs.append(pickle.load(inp).pd_TRAPS_df[['time', 'origin_ID', 
                                                             'vircle', 'phase_shift', 
                                                             'pattern_flag_NBV', 'configu_flag_NBV', 'qorder_flag_NBV']].copy())
    
    
    stop_timer = time.perf_counter()
    print('loaded ' + pkl_VIRCLES_importname + f' in: {stop_timer - start_timer:0.1f} seconds')
    
# stack the yearly dataframes
pd_VIRCLES_df = pd.concat(pd_VIRCLES_sdfs, copy=True)

# reset the index after stacking
pd_VIRCLES_df.reset_index(drop=True, inplace=True)

# save memory
del pd_VIRCLES_sdfs

loaded MULTIOBS_24HI_e025_TRAPS_GPGP_2000.pkl in: 5.4 seconds
loaded MULTIOBS_24HI_e025_TRAPS_GPGP_2001.pkl in: 4.8 seconds
loaded MULTIOBS_24HI_e025_TRAPS_GPGP_2002.pkl in: 4.8 seconds
loaded MULTIOBS_24HI_e025_TRAPS_GPGP_2003.pkl in: 4.6 seconds
loaded MULTIOBS_24HI_e025_TRAPS_GPGP_2004.pkl in: 4.7 seconds
loaded MULTIOBS_24HI_e025_TRAPS_GPGP_2005.pkl in: 4.5 seconds
loaded MULTIOBS_24HI_e025_TRAPS_GPGP_2006.pkl in: 4.8 seconds
loaded MULTIOBS_24HI_e025_TRAPS_GPGP_2007.pkl in: 4.8 seconds
loaded MULTIOBS_24HI_e025_TRAPS_GPGP_2008.pkl in: 4.4 seconds
loaded MULTIOBS_24HI_e025_TRAPS_GPGP_2009.pkl in: 4.5 seconds
loaded MULTIOBS_24HI_e025_TRAPS_GPGP_2010.pkl in: 4.9 seconds
loaded MULTIOBS_24HI_e025_TRAPS_GPGP_2011.pkl in: 4.9 seconds
loaded MULTIOBS_24HI_e025_TRAPS_GPGP_2012.pkl in: 5.0 seconds
loaded MULTIOBS_24HI_e025_TRAPS_GPGP_2013.pkl in: 4.6 seconds
loaded MULTIOBS_24HI_e025_TRAPS_GPGP_2014.pkl in: 4.9 seconds
loaded MULTIOBS_24HI_e025_TRAPS_GPGP_2015.pkl in: 4.7 seconds
loaded M

In [12]:
# print check
pd_VIRCLES_df

,time,origin_ID,vircle,phase_shift,pattern_flag_NBV,configu_flag_NBV,qorder_flag_NBV
0,2000-01-01,200001010000 000,False,3.609645,99,Z,99
1,2000-01-01,200001010000 001,False,5.546844,99,Z,99
2,2000-01-01,200001010000 002,False,5.048653,99,Z,99
3,2000-01-01,200001010000 003,True,3.524186,6,B,0
4,2000-01-01,200001010000 004,True,3.142709,10,F,2
...,...,...,...,...,...,...,...
4076060,2019-12-31,201912100000 161,True,6.135567,6,B,0
4076061,2019-12-31,201912300000 423,True,4.501250,2,E,1
4076062,2019-12-31,201912290000 544,True,0.561322,5,E,1
4076063,2019-12-31,201912290000 161,True,3.951574,6,B,0


## Reduce VIRCLES dataframe to PROXIMITY originIDs and timestamp dates

From the VIRCLES dataframe, we only need the TRAP instances that also appear in the PROXIMITY dataframe.

In [13]:
# get all timestamps, this contains duplicates, use the DatetimeIndex object type
timestamps_PROXIMITY = pd.to_datetime(pd_PROXIMITY_df.time.to_numpy(copy=True))
timestamp_dates_PROXIMITY = timestamps_PROXIMITY.date
timestamp_dates_PROXIMITY_unique = np.unique(timestamps_PROXIMITY.date)

# and get all TRAPS IDs that occur in the PROXIMITY dataframe
origin_IDs_PROXIMITY = pd_PROXIMITY_df.TRAP_ID.to_numpy(copy=True)
origin_IDs_PROXIMITY_unique = np.unique(origin_IDs_PROXIMITY)

# further below we assert that the PTRT and PROXIMITY dataframes contain the same set of origin IDs

In [14]:
start_timer = time.perf_counter()

In [15]:
# building a filter using dictionaries is thousand times faster than adding boolean arrays
originIDs_filter_DICT = {}
timestampdates_filter_DICT = {}

for coriginID in origin_IDs_PROXIMITY_unique:
    originIDs_filter_DICT[coriginID] = True

for cdate in timestamp_dates_PROXIMITY_unique:
    timestampdates_filter_DICT[cdate] = True
    
# use the pandas map() function to build the filter
# assigns True to every PTRT origin ID in the VIRCLES dataframe and NaN(-> False) to all others
originIDs_filter = pd_VIRCLES_df.origin_ID.map(originIDs_filter_DICT).fillna(False).to_numpy(copy=True)
timestampdates_filter = pd_VIRCLES_df.time.dt.date.map(timestampdates_filter_DICT).fillna(False).to_numpy(copy=True)

# crop dataframe
pd_VIRCLES_df = pd_VIRCLES_df[(originIDs_filter & timestampdates_filter)].copy()

# reset the index after cropping
pd_VIRCLES_df.reset_index(drop=True, inplace=True)

In [16]:
origin_IDs_PTRT = pd_PTRT_df.origin_ID.to_numpy(copy=True)

# assert that all dataframes bear the same origin IDs
# mind that one origin ID can appear multiple times in the PTRT dataframe!
assert np.all(np.unique(origin_IDs_PTRT)==np.unique(pd_PROXIMITY_df.TRAP_ID)), 'PTRT and PROXIMITY origin IDs do not coincide'
assert np.all(np.unique(origin_IDs_PTRT)==np.unique(pd_VIRCLES_df.origin_ID)), 'PTRT and TRAPS origin IDs do not coincide'

In [17]:
stop_timer = time.perf_counter()
print(f'task time: {stop_timer - start_timer:0.4f} seconds')

task time: 2.2802 seconds


In [18]:
# print check
pd_VIRCLES_df

,time,origin_ID,vircle,phase_shift,pattern_flag_NBV,configu_flag_NBV,qorder_flag_NBV
0,2000-01-01,200001010000 055,True,5.602028,14,I,1
1,2000-01-01,200001010000 056,False,4.583939,99,Z,99
2,2000-01-01,200001010000 057,True,2.872863,6,B,0
3,2000-01-01,200001010000 058,True,1.746609,6,B,0
4,2000-01-01,200001010000 059,False,3.023511,99,Z,99
...,...,...,...,...,...,...,...
244369,2019-12-31,201912250000 548,True,1.803174,6,B,0
244370,2019-12-31,201912170000 244,True,1.493969,6,B,0
244371,2019-12-31,201912220000 450,True,5.483759,6,B,0
244372,2019-12-31,201912180000 485,True,5.257449,6,B,0


# Enhance proximity dataframe

Enhance the PROXIMITY dataframe with characteristics from the TRAPS VIRCLES dataframe.

## Introduce columns for additional TRAP characteristics

In [19]:
# use this to initialise column arrays of the right size
pd_PROXIMITY_df['vircle'] = False
pd_PROXIMITY_df['phase_shift'] = np.nan
pd_PROXIMITY_df['pattern_flag_NBV'] = np.nan
pd_PROXIMITY_df['configu_flag_NBV'] = ''
pd_PROXIMITY_df['qorder_flag_NBV'] = np.nan

## Columns to arrays

In [20]:
# get all timestamps, this contains duplicates, use the DatetimeIndex object type
timestamps_VIRCLES = pd.to_datetime(pd_VIRCLES_df.time.to_numpy(copy=True))
timestamps_PROXIMITY = pd.to_datetime(pd_PROXIMITY_df.time.to_numpy(copy=True))

timestamp_dates_VIRCLES = timestamps_VIRCLES.date
timestamp_dates_PROXIMITY = timestamps_PROXIMITY.date

In [21]:
origin_IDs_VIRCLES = pd_VIRCLES_df.origin_ID.to_numpy(copy=True)
vircles_VIRCLES = pd_VIRCLES_df.vircle.to_numpy(copy=True)
phase_shifts_VIRCLES = pd_VIRCLES_df.phase_shift.to_numpy(copy=True)
pattern_flags_VIRCLES = pd_VIRCLES_df.pattern_flag_NBV.to_numpy(copy=True)
configu_flags_VIRCLES = pd_VIRCLES_df.configu_flag_NBV.to_numpy(copy=True)
qorder_flags_VIRCLES = pd_VIRCLES_df.qorder_flag_NBV.to_numpy(copy=True)

origin_IDs_PROXIMITY = pd_PROXIMITY_df.TRAP_ID.to_numpy(copy=True)
vircles_PROXIMITY = pd_PROXIMITY_df.vircle.to_numpy(copy=True)
phase_shifts_PROXIMITY = pd_PROXIMITY_df.phase_shift.to_numpy(copy=True)
pattern_flags_PROXIMITY = pd_PROXIMITY_df.pattern_flag_NBV.to_numpy(copy=True)
configu_flags_PROXIMITY = pd_PROXIMITY_df.configu_flag_NBV.to_numpy(copy=True)
qorder_flags_PROXIMITY = pd_PROXIMITY_df.qorder_flag_NBV.to_numpy(copy=True)

# we will need the following PROXIMITY columns later
pair_IDs_PROXIMITY = pd_PROXIMITY_df.pair_ID.to_numpy(copy=True)
TRAP_lifetimes_PROXIMITY = pd_PROXIMITY_df.lifetime.to_numpy(copy=True)
TRAP_ages_PROXIMITY = pd_PROXIMITY_df.age.to_numpy(copy=True)
TRAP_core_attractions_PROXIMITY = pd_PROXIMITY_df.core_attraction.to_numpy(copy=True)
core_distances_PROXIMITY = pd_PROXIMITY_df.distance_to_core.to_numpy(copy=True)
curve_distances_PROXIMITY = pd_PROXIMITY_df.distance_to_curve.to_numpy(copy=True)
proximity_times_PROXIMITY = pd_PROXIMITY_df.proximity_time.to_numpy(copy=True)
proximity_ages_PROXIMITY = pd_PROXIMITY_df.proximity_age.to_numpy(copy=True)

## Transfer data from VIRCLES to PROXIMITY

Assign data from the VIRCLES to the PROXIMITY dataframe.  
This takes a lot of time!

In [22]:
start_timer = time.perf_counter()

In [23]:
number_of_drifter_positions = timestamps_PROXIMITY.size

# shortcut for test runs
row_indices=range(100) if notebook_run else range(number_of_drifter_positions)

for row_index in row_indices: # iterate through all drifter positions
        
    # this filter is unique
    # as we also want to compare 1HI and 6HI drifter data against 24HI TRAPS data, 
    # we have to compare dates instead of the full timestamps
    #closest_TRAP_filter = ((timestamp_dates_VIRCLES==timestamp_dates_PROXIMITY[row_index]) & 
    #                       (origin_IDs_VIRCLES==origin_IDs_PROXIMITY[row_index]))

    # this is faster
    closest_TRAP_index = np.argmax((timestamp_dates_VIRCLES==timestamp_dates_PROXIMITY[row_index]) & 
                                   (origin_IDs_VIRCLES==origin_IDs_PROXIMITY[row_index]))
    
    vircles_PROXIMITY[row_index] = vircles_VIRCLES[closest_TRAP_index]
    phase_shifts_PROXIMITY[row_index] = phase_shifts_VIRCLES[closest_TRAP_index]
    pattern_flags_PROXIMITY[row_index] = pattern_flags_VIRCLES[closest_TRAP_index]
    configu_flags_PROXIMITY[row_index] = configu_flags_VIRCLES[closest_TRAP_index]
    qorder_flags_PROXIMITY[row_index] = qorder_flags_VIRCLES[closest_TRAP_index]
    
    # show progress in terminal
    if not notebook_run:
        print('enhanced proximity dataframe at drifter position ' 
              + str(row_index).zfill(len(str(number_of_drifter_positions-1))) + '/' + str(number_of_drifter_positions-1))

    
# update the overall dataframe
pd_PROXIMITY_df['vircle'] = vircles_PROXIMITY
pd_PROXIMITY_df['phase_shift'] = phase_shifts_PROXIMITY
pd_PROXIMITY_df['pattern_flag_NBV'] = pattern_flags_PROXIMITY
pd_PROXIMITY_df['configu_flag_NBV'] = configu_flags_PROXIMITY
pd_PROXIMITY_df['qorder_flag_NBV'] = qorder_flags_PROXIMITY

In [24]:
stop_timer = time.perf_counter()
print(f'task time: {stop_timer - start_timer:0.4f} seconds')

task time: 1.0041 seconds


In [25]:
# print check
pd_PROXIMITY_df.columns

Index(['drifter_ID', 'time', 'drifter_lat', 'drifter_lon', 'drifter_U',
       'drifter_V', 'drogued', 'TRAP_ID', 'distance_to_core', 'core_lon',
       'core_lat', 'core_attraction', 'distance_to_curve',
       'closest_curvepoint_index', 'on_curve_25km', 'on_curve_50km',
       'curve_lons', 'curve_lats', 'curve_attractions', 'lifetime', 'age',
       'pair_ID', 'proximity_time', 'proximity_age', 'drifter_vector_lon',
       'drifter_vector_lat', 'drifter_angle_DEG', 'drifter_vector_zonal',
       'drifter_vector_meridional', 'drifter_angle_KM', 'vircle',
       'phase_shift', 'pattern_flag_NBV', 'configu_flag_NBV',
       'qorder_flag_NBV'],
      dtype='object')

# Determine reachable pairs

We analyse for every drifter-TRAP pair if the given TRAP remains long enough to be reached by the drifter before it dissipates.  
To approximate this, we divide the initial drifter distance to the TRAP core by a drifter velocity of 17 km/day, i.e. the average drifter
speed in our dataset. If the resulting number of days is less or equal than the remaining lifetime of the TRAP at first encounter, the drifter-TRAP pair is considered *reachable*.  
Remaining lifetime = lifetime - age. This automatically sorts out all ephemeral TRAPs with lifetime=1 day.

From this we may derive how many drifter days are related to reachable drifter-TRAP pairs, i.e. are spent within a reachbale distance around a sufficiently persistent
TRAP. From these measures we get an impression of how available drifters are to TRAP attraction.

In [26]:
# define remaining TRAP lifetime
TRAP_remaining_lifetimes_PROXIMITY = TRAP_lifetimes_PROXIMITY - TRAP_ages_PROXIMITY

pd_PROXIMITY_df['remaining_lifetime'] = TRAP_remaining_lifetimes_PROXIMITY

# assert that there are no negative values
assert np.all(TRAP_remaining_lifetimes_PROXIMITY>=0), 'found negative remaining lifetime'

In [27]:
# and define the days a drifter would need on average to overcome the current distance to core or curve
# we compute this for all drifter positions, however later we will only use this value at first encounter of a drifter-TRAP pair
days_to_reach_core_PROXIMITY = core_distances_PROXIMITY / 17 # 17 km/day is the average drifter speed in our dataset
days_to_reach_curve_PROXIMITY = curve_distances_PROXIMITY / 17 # 17 km/day is the average drifter speed in our dataset

pd_PROXIMITY_df['days_to_reach_core'] = days_to_reach_core_PROXIMITY
pd_PROXIMITY_df['days_to_reach_curve'] = days_to_reach_curve_PROXIMITY

In [28]:
# now define if for a given drifter position, the closest TRAP remains long enough to be reached,
# i.e. determine if the days to reach core OR curve are less equal than the remaining lifetime
# if this is given, the drifter-TRAP pair instance is reachable
# again, apply this to all positions even though only the value at first encounter will be needed later
reachable_core_filter = (days_to_reach_core_PROXIMITY<=TRAP_remaining_lifetimes_PROXIMITY)
reachable_curve_filter = (days_to_reach_curve_PROXIMITY<=TRAP_remaining_lifetimes_PROXIMITY)

reachables_PROXIMITY = (reachable_core_filter | reachable_curve_filter)

pd_PROXIMITY_df['reachable'] = reachables_PROXIMITY

In [29]:
# print check
pd_PROXIMITY_df[['lifetime', 'age', 'remaining_lifetime', 'days_to_reach_core', 'days_to_reach_curve', 'reachable']]

,lifetime,age,remaining_lifetime,days_to_reach_core,days_to_reach_curve,reachable
0,3,2,1,3.543012,2.632559,False
1,3,3,0,2.683813,2.331337,False
2,1,1,0,3.735484,2.134851,False
3,20,14,6,3.761175,2.732694,True
4,20,15,5,3.088237,2.852254,True
...,...,...,...,...,...,...
57558,15,4,11,2.303581,2.238955,True
57559,15,5,10,1.921043,1.430022,True
57560,15,6,9,2.457121,1.286479,True
57561,15,7,8,3.161686,0.161962,True


# Create hyperbolic pair statistics dataframe

Now take the PTRT and for every drifter-TRAP pair ID add metrics like 
- the TRAP lifetime  
- the minimum TRAP age  
- the reachable flag at first encounter  
- an indicator if vircles are always available
- an array of the core attractions they involve  
- (an array of the vorticity patterns they involve)
- (an array of the vorticity configurations they involve)
- an array of the vorticity quadrupole orders they involve
- an array of the phase shifts of the individual TRAP instances


In [30]:
# prefer to create a new dataframe and keep the original PTRT one
pd_HPS_df = pd_PTRT_df.copy()

In [31]:
# we define the unit of the retention time as days
pd_HPS_df.rename(columns={'counts': 'retention_time'}, inplace=True)

## Introduce columns for additional TRAP characteristics

For some attributes, already create the columns.  
We can only make statements about vorticity patterns during a proximity time when there have been vircles computed at every TRAP instance of a drifter-TRAP pair.

In [32]:
# use this to initialise column arrays of the right size
pd_HPS_df['TRAP_lifetime'] = 999
pd_HPS_df['minimum_TRAP_age'] = 999
pd_HPS_df['reachable'] = False
pd_HPS_df['vircles_always_available'] = False
# more other columns will be created at runtime

In [33]:
# print check
pd_HPS_df

,pair_ID,origin_ID,retention_time,proximity_time,TRAP_lifetime,minimum_TRAP_age,reachable,vircles_always_available
0,201310310000 000000000101916 201310270000 344,201310270000 344,46,1104,999,999,False,False
1,201905270000 300234066316870 201905150000 291,201905150000 291,45,1080,999,999,False,False
2,201507280000 000000000132718 201507280000 374,201507280000 374,44,1056,999,999,False,False
3,200010120000 000000009729746 200010120000 069,200010120000 069,41,984,999,999,False,False
4,201102160000 000000000093582 201101310000 284,201101310000 284,39,936,999,999,False,False
...,...,...,...,...,...,...,...,...
16804,200912120000 000000000079281 200912100000 524,200912100000 524,1,24,999,999,False,False
16805,200912130000 000000000079271 200911250000 277,200911250000 277,1,24,999,999,False,False
16806,200912130000 000000000079281 200911250000 268,200911250000 268,1,24,999,999,False,False
16807,200912170000 000000000079281 200912170000 581,200912170000 581,1,24,999,999,False,False


## Columns to arrays

In [34]:
pair_IDs_HPS = pd_HPS_df.pair_ID.to_numpy(copy=True)
retention_times_HPS = pd_HPS_df.retention_time.to_numpy(copy=True)

# new columns
TRAP_lifetimes_HPS = pd_HPS_df.TRAP_lifetime.to_numpy(copy=True)
minimum_TRAP_ages_HPS = pd_HPS_df.minimum_TRAP_age.to_numpy(copy=True)
reachables_HPS = pd_HPS_df.reachable.to_numpy(copy=True)
vircles_always_availables_HPS = pd_HPS_df.vircles_always_available.to_numpy(copy=True)

# and initiate simple lists to collect more data
# involved_patterns = []
# involved_configus = []
involved_attractions = []
involved_qorders = []
involved_phase_shifts = []

## Transfer data from PROXIMITY to HPS

In [35]:
# assign statistics from the proximity to the HPS dataframe
for ix in range(pair_IDs_HPS.size): # iterate through all drifter-TRAP pairs
    
    # current pair ID filter that fits the PROXIMITY dataframe
    cpairID_filter = (pair_IDs_PROXIMITY==pair_IDs_HPS[ix])
    
    # assert that the current pair instances are sorted in time
    assert np.all(timestamps_PROXIMITY[cpairID_filter]==np.sort(timestamps_PROXIMITY[cpairID_filter])), 'unsorted pair instances'    
    
    # extract data from the column arrays of the PROXIMITY dataframe
    TRAP_lifetimes_HPS[ix] = np.min(TRAP_lifetimes_PROXIMITY[cpairID_filter]) # max() should give the same
    minimum_TRAP_ages_HPS[ix] = np.min(TRAP_ages_PROXIMITY[cpairID_filter])
    reachables_HPS[ix] = reachables_PROXIMITY[cpairID_filter][0] # the flag at first encounter
    
    # this checks at every drifter timestep which can be smaller than the 24H TRAPS timestep
    vircles_always_availables_HPS[ix] = True if np.all(vircles_PROXIMITY[cpairID_filter]) else False    

    # involved_patterns.append(pattern_flags_PROXIMITY[cpairID_filter])
    # involved_configus.append(configu_flags_PROXIMITY[cpairID_filter])
    involved_attractions.append(TRAP_core_attractions_PROXIMITY[cpairID_filter])
    involved_qorders.append(qorder_flags_PROXIMITY[cpairID_filter])
    involved_phase_shifts.append(phase_shifts_PROXIMITY[cpairID_filter])
    

# update the overall dataframe
pd_HPS_df['TRAP_lifetime'] = TRAP_lifetimes_HPS
pd_HPS_df['minimum_TRAP_age'] = minimum_TRAP_ages_HPS
pd_HPS_df['reachable'] = reachables_HPS
pd_HPS_df['vircles_always_available'] = vircles_always_availables_HPS
# pd_HPS_df['involved_patterns'] = involved_patterns
# pd_HPS_df['involved_configus'] = involved_configus
pd_HPS_df['involved_attractions'] = involved_attractions
pd_HPS_df['involved_qorders'] = involved_qorders
pd_HPS_df['involved_phase_shifts'] = involved_phase_shifts

In [36]:
# print check
# MIND THAT WE HAVE ONLY TRANSFERED DATA FOR 100 DRIFTERS
pd_HPS_df[vircles_always_availables_HPS].involved_qorders
sum(pd_HPS_df.reachable)/pd_HPS_df.size
pd_HPS_df

,pair_ID,origin_ID,retention_time,proximity_time,TRAP_lifetime,minimum_TRAP_age,reachable,vircles_always_available,involved_attractions,involved_qorders,involved_phase_shifts
0,201310310000 000000000101916 201310270000 344,201310270000 344,46,1104,93,5,True,False,"[-0.139247397665006, -0.133322592384704, -0.14...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
1,201905270000 300234066316870 201905150000 291,201905150000 291,45,1080,81,13,True,False,"[-0.20999680205224, -0.196616953730887, -0.196...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2,201507280000 000000000132718 201507280000 374,201507280000 374,44,1056,57,1,True,False,"[-0.201265758720064, -0.19937420075606, -0.181...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
3,200010120000 000000009729746 200010120000 069,200010120000 069,41,984,88,1,True,False,"[-0.181595150679058, -0.205832111166168, -0.20...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
4,201102160000 000000000093582 201101310000 284,201101310000 284,39,936,55,17,True,False,"[-0.107347613092045, -0.111451057209841, -0.12...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
...,...,...,...,...,...,...,...,...,...,...,...
16804,200912120000 000000000079281 200912100000 524,200912100000 524,1,24,3,3,False,False,[-0.147340091322843],[nan],[nan]
16805,200912130000 000000000079271 200911250000 277,200911250000 277,1,24,36,19,True,False,[-0.224042379138215],[nan],[nan]
16806,200912130000 000000000079281 200911250000 268,200911250000 268,1,24,45,19,True,False,[-0.221316642693239],[nan],[nan]
16807,200912170000 000000000079281 200912170000 581,200912170000 581,1,24,1,1,False,False,[-0.110832275495029],[nan],[nan]


## Normalise retention ages

This can be done super easily when knowing the total retention time.

In [37]:
# normalisation is relative since the first age is always 1 day
# nage = (age - 1 day) / (lifetime - 1 day), division by zero is prevented by only taking 3 instance pairs
retention_nages_HPS = [(np.arange(retention_time)/(retention_time-1)) if retention_time>2 else np.array([]) for retention_time in retention_times_HPS]

pd_HPS_df['retention_nages'] = retention_nages_HPS

In [38]:
# print check
# pd_HPS_df
# retention_nages_HPS

## Get the original core and curve distances

In [39]:
original_core_distances = []
original_curve_distances = []

for ix in range(pair_IDs_HPS.size): # iterate through all drifter-TRAP pairs
               
    # current pair ID filter that fits the PROXIMITY dataframe
    cpairID_filter = (pair_IDs_PROXIMITY==pair_IDs_HPS[ix])
    
    # append the characteristics of the current pair
    original_core_distances.append(core_distances_PROXIMITY[cpairID_filter])
    original_curve_distances.append(curve_distances_PROXIMITY[cpairID_filter])
            
        
# update the overall dataframe
pd_HPS_df['original_core_distances'] = original_core_distances
pd_HPS_df['original_curve_distances'] = original_curve_distances

In [40]:
# print check
# pd_HPS_df
# original_curve_distances

# Export the 20-years pickle file

In [41]:
# create the object, using the former TRAPS object
HPS_data = TRAPSdata(vel_product_short, vel_product_long, pd_HPS_df)

In [42]:
# save the object as .pkl file
start_timer = time.perf_counter()

pkl_HPS_exportname = vel_product_short + epsilon_string + '_' + gdp_product_short + drogue_state
pkl_HPS_exportname += '_TRAPS_DRIFTERS_HPS_0019' + rho_kmstring + '.pkl'

# save the object as .pkl file                
save_object(HPS_data, pkl_HPS_exportpath + pkl_HPS_exportname)

stop_timer = time.perf_counter()

print('saved ' + pkl_HPS_exportname + f' in: {stop_timer - start_timer:0.1f} seconds')

saved MULTIOBS_24HI_e025_GDP_24HI_DROGUED_TRAPS_DRIFTERS_HPS_0019_r075.pkl in: 0.4 seconds


## End sound

In [43]:
# measure the computation time for the entire script
stop_script_timer = time.perf_counter()
print(f'overall computation time: {stop_script_timer - start_script_timer:0.3f} seconds')

overall computation time: 139.633 seconds


In [44]:
#https://gist.github.com/tamsanh/a658c1b29b8cba7d782a8b3aed685a24

framerate = 4410
play_time_seconds = 1

t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
# G-Dur
#audio_data = np.sin(2*np.pi*391*t) + np.sin(2*np.pi*493*t) + np.sin(2*np.pi*587*t)
# D-Dur
audio_data = np.sin(2*np.pi*293*t) + np.sin(2*np.pi*369*t) + np.sin(2*np.pi*440*t)
Audio(audio_data, rate=framerate, autoplay=True)